In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error

import tensorflow as tf
keras = tf.keras
from datetime import datetime

from typing import List

import os
import glob

from src.io.load import DataLoader

from dataclasses import dataclass, field
from typing import Dict
from src.action_strategy.abstract_models import ActionStrategy
from src.action_strategy.mean_reversion import MeanReversion
from src.forecast.factory import ModelForecastFactory
from src.forecast.runner import ForecastRunner
from src.forecast.utils import SamplerSetForecast
from src.io.load import DataLoader, load_config
from src.io.save import Dumper
from src.preprocessing.preprocess import ProcessData
from src.stock import Balance, Stock
import logging
from src.trade_manager import TradeManagerV0
from datetime import datetime, timedelta
import math
from abc import ABC, abstractmethod
import pprint as pp
import copy

In [3]:
PATH_DATA_RESULTS = "results/"

In [4]:
import logging
from src.main import load_config, load_data, Experiment

In [5]:
logging.basicConfig()
log=logging.getLogger(__name__)
log.setLevel(logging.DEBUG)

### Data

In [6]:
batch_name = "full_year_germany_20221108_204632"
trading_data = pd.read_excel(os.path.join(PATH_DATA_RESULTS, batch_name, f"results_{batch_name}.xlsx"), index_col=0)

In [7]:
trading_data

,true_prices,price_flow,action,current_balance,energy_trade,current_cpty,mean,std,upper_bollinger,lower_bollinger,pred_prices
2020-01-01 13:00:00,30.75,-3075.000,BUY,-2075.000000,-100.00,86.0,35.846957,6.166914,40.163796,31.530117,30.784292
2020-01-01 14:00:00,32.11,0.000,HOLD,-2075.000000,0.00,86.0,34.739130,5.651647,38.695283,30.782978,32.850411
2020-01-01 15:00:00,35.98,0.000,HOLD,-2075.000000,0.00,86.0,34.228261,4.905835,37.662345,30.794177,35.649802
2020-01-01 16:00:00,40.40,2987.984,SELL,912.984000,73.96,0.0,33.984783,4.408833,37.070966,30.898600,37.334166
2020-01-01 17:00:00,44.05,0.000,HOLD_s,912.984000,0.00,0.0,34.065217,4.579112,37.270595,30.859839,37.603027
...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 08:00:00,48.19,0.000,HOLD,468439.677041,0.00,930.0,45.404348,9.866615,52.310978,38.497717,38.718738
2020-12-31 09:00:00,43.23,0.000,HOLD,468439.677041,0.00,930.0,45.025217,9.770731,51.864729,38.185705,39.595432
2020-12-31 10:00:00,43.57,0.000,HOLD,468439.677041,0.00,930.0,44.500870,9.495479,51.147705,37.854034,40.263511
2020-12-31 11:00:00,50.52,0.000,HOLD,468439.677041,0.00,930.0,44.346522,9.360761,50.899054,37.793989,40.005251


### Test with example data (1 day)

In [30]:
dummy_index = trading_data[11:35].index

In [31]:
dummy_index

Index(['2020-01-02 00:00:00', '2020-01-02 01:00:00', '2020-01-02 02:00:00',
       '2020-01-02 03:00:00', '2020-01-02 04:00:00', '2020-01-02 05:00:00',
       '2020-01-02 06:00:00', '2020-01-02 07:00:00', '2020-01-02 08:00:00',
       '2020-01-02 09:00:00', '2020-01-02 10:00:00', '2020-01-02 11:00:00',
       '2020-01-02 12:00:00', '2020-01-02 13:00:00', '2020-01-02 14:00:00',
       '2020-01-02 15:00:00', '2020-01-02 16:00:00', '2020-01-02 17:00:00',
       '2020-01-02 18:00:00', '2020-01-02 19:00:00', '2020-01-02 20:00:00',
       '2020-01-02 21:00:00', '2020-01-02 22:00:00', '2020-01-02 23:00:00'],
      dtype='object')

In [32]:
#l_prices = [48, 48, 47, 47, 48, 61, 73, 79, 76, 72, 66, 60, 58, 59, 57, 55, 60, 55, 51, 47, 46, 43, 38, 37]
l_prices = [47.94, 47.67, 46.6, 46.72, 47.58, 61.19, 73.06, 79, 75.65, 72.03, 66.39, 60.06, 57.7, 59.41, 57.39, 54.54, 59.94, 55.08, 51.42, 46.97, 45.63, 42.95, 37.83, 36.68]

In [33]:
len(l_prices)

24

In [34]:
prices = pd.DataFrame({"value": l_prices, "true_prices":l_prices}, index=dummy_index)

### Test with real data

In [48]:
#prices.index = np.arange(0, 24)
#prices.index = dummy_index
#prices = prices.to_dict()
file = os.path.join("results", "full_year_pair_method_germany_20221203_165353", "results_full_year_pair_method_germany_20221203_165353.xlsx")
prices_df = pd.read_excel(file, index_col=0)

In [65]:
prices = prices_df[24*2:24*3]
prices = prices[["true_prices", "pred_prices"]].rename({"pred_prices":"value"}, axis=1)

In [66]:
prices

,true_prices,value
2020-01-03 00:00:00,13.04,18.400147
2020-01-03 01:00:00,1.53,15.482259
2020-01-03 02:00:00,0.14,12.918863
2020-01-03 03:00:00,0.85,12.865515
2020-01-03 04:00:00,9.92,13.802502
2020-01-03 05:00:00,24.48,16.094563
2020-01-03 06:00:00,26.68,20.752541
2020-01-03 07:00:00,28.81,24.541543
2020-01-03 08:00:00,29.28,29.486725
2020-01-03 09:00:00,28.85,32.287523


#### Recap

* Find position and value of highest and lower
    * buy 1 and sell 0.7
    
    
#### Object needed
* Passes : dictionary
    * still_cpty : float 
    * passes : list[dict]
        * prices : dict 
        * trading : dict
            * qty
            * value
            * type
                * sBb : sell before buy
                * bBs : buy before sell
* Stock

### Init bases classes

In [67]:
from src.action_strategy.abstract_models import TradeAction

In [68]:
from src.stock import Stock, Balance

config_file = "1"
# Load config and data
log.info("Load data and config..")
config_exp = load_config(config_file)
log.info(f"""LET'S TRADE ON {config_exp["market_name"]}""")
config_exp["date_start"] =  datetime.strptime(' '.join([config_exp["date_start"], config_exp["hour_begin_trade"]]), "%d-%m-%Y %H")
config_exp["date_end"] =  datetime.strptime(' '.join([config_exp["date_end"], config_exp["hour_begin_trade"]]), "%d-%m-%Y %H")

config_exp["n_iteration"]  = (config_exp["date_end"] - config_exp["date_start"]).days

#prices = load_data(config_exp["market_name"], end_year=config_exp["date_end"])
    
stock = Stock(storage_pwr=config_exp["storage_pwr"], 
                rho_d=config_exp["rho_d"], 
                rho_s=config_exp["rho_s"],
                t_discharge=config_exp["t_discharge"],
                init_storage_cpty=config_exp["init_storage_cpty"],
                )

balance = Balance(init_level=config_exp["balance_init_level"])

INFO:__main__:Load data and config..
INFO:__main__:LET'S TRADE ON germany


## Modif Stock
* allow qty passing

In [69]:
@dataclass
class Stock:
    storage_pwr: float
    rho_d: float
    rho_s: float
    t_discharge: float
    init_storage_cpty: float
    current_cpty: float = field(init=False)
    storage_cpty: float = field(init=False)


    def __post_init__(self):
        self.init_stock()
        self.history = []

    def init_stock(self):
        self.storage_cpty = int(self.storage_pwr*(self.t_discharge/self.rho_d))
        self.current_cpty = self.init_storage_cpty
    
    def update_stock(self, action: str, flow:float=None):
        """
        we have equivalence in energy trade unit bewteen what we give to market
        """
        #print(f"action Stock : {action}")
        qty=0
        if action == TradeAction.BUY.name:
            qty = self.add_to_stock(flow)
        elif action == TradeAction.SELL.name:
            qty = self.retrieve_from_stock(flow)
        print(f"Updated Stock : {self.current_cpty}")
        self.history.append((qty, self.current_cpty,))
        return qty

    @property
    def is_empty(self):
        return self.current_cpty == 0
    
    @property
    def is_full(self):
        return self.current_cpty == self.storage_cpty

    def add_to_stock(self, flow=None):
        """BUY"""
        if flow is None:
            qty = (
                min(
                    (self.storage_cpty - self.current_cpty), 
                    (self.storage_pwr*self.rho_s*1)
                )
            if not self.is_full else 0.)
            # flow is what energy is bought from market : coef for price (negative)

            # normal buy - add rho_s*qty to stock but bought self.storage_pwr*1
            if (qty/self.rho_s/1 == self.storage_pwr):
                flow = -self.storage_pwr*1
            else:
            # almost full; to add cpty we need to buy qty/rho_s
                flow = -qty/self.rho_s
        else:
            qty = min(flow*self.rho_s, (self.storage_cpty - self.current_cpty))
            flow=-flow
            
        print(f'current : {self.current_cpty} - add qty : {qty} - flow : {flow}')
        self.current_cpty += qty
        

        return flow


    def retrieve_from_stock(self, flow=None):
        """SELL"""
        if flow is None:
            qty = (
                min(
                    self.current_cpty, 
                    (self.storage_pwr/self.rho_d*1)
                )
            if not self.is_empty else 0.)

            # flow is what energy is given to market : coef for price (positive)

            # normal sell : we retrieve
            if (qty*self.rho_d/1 == self.storage_pwr):
                # flow to market after rho_d
                flow = self.storage_pwr*1
            else:
                # almost empty we give to market what we can : qty*rho_d
                flow = qty*self.rho_d
        else:
            qty = min(flow/self.rho_d, self.current_cpty)

        
        print(f'current : {self.current_cpty} - retrieve qty : {qty} - flow : {flow}')
        self.current_cpty -= qty

        return flow
    
@dataclass
class Balance:
    init_level: float

    def __post_init__(self):
        self.init_balance()

    def init_balance(self):
        self.current_level = self.init_level
        self.history = []

    def update_balance(self, action: str, price: float, flow: float):
        # check if stock was updated
        self.current_level += price*flow
        log.info(f"ADD : {price*flow} €")
        log.info(f"Current : { self.current_level} €")

        action = action if flow !=0 else (f"HOLD_{action[0].lower()}") if action != "HOLD" else "HOLD"
        self.history.append((price, price*flow, action, self.current_level,))
        return self

sell partially

In [70]:
from dataclasses import dataclass
@dataclass
class Price:
    idx:int
    value:float
    type_order:str

In [157]:
class CheckOrderValidity:
    """check validity order
    - sequential daily order found
    - return idx's day when stock is full or empty
    """

    def __init__(self):
        pass

    def trade_unit(self, s_price: pd.Series, stock, balance):
        # print("----------")
        # print(f"IDX {s_price.name}")
        flow = stock.update_stock(s_price["action"], s_price["flow"])

        balance.update_balance(
            action=s_price["action"],
            price=s_price["true_prices"],
            flow=flow,
        )
        return flow, balance, stock

    def check_orders_validty(self, orders, stock, balance):
        flows = []
        stock_c, balance_c = copy.deepcopy(stock), copy.deepcopy(balance)

        orders["true_prices"] = orders["value"]
        print("\n ==== Validity Check ==== \n")
        for idx, s_price in orders.iterrows():
            print(f"=== {idx} ===")
            print(f"Current cpty : {stock_c.current_cpty}")
            print(f"Current balance : {balance_c.current_level}")

            flow, balance_c, stock_c = self.trade_unit(s_price, stock_c, balance_c)
            print(s_price['action'], flow)
            if stock_c.is_empty or stock_c.is_full:
                return (
                    dict(idx=idx, reason="empty")
                    if stock_c.is_empty
                    else dict(idx=idx, reason="full")
                )
            flows.append(flow)

        return dict(idx=idx, reason="valid")


class PairMethod(ActionStrategy):
    """
    Pair method strategy
    """

    def __init__(self, storage_pwr, rho_d, rho_s, init_storage_cpty, **kwargs):
        self.unit_pwr = storage_pwr
        self.rho_d = rho_d
        self.rho_s = rho_s
        self.night_stock = init_storage_cpty
        self.validity_checker = CheckOrderValidity()
        self.kwargs = kwargs

    def get_indicator(self, x_train=None, x_test=None):
        """dummy function for TradeManager integration"""

    def get_action_window(
        self, window_prices: pd.DataFrame, stock, balance
    ) -> pd.DataFrame:
        """deploy pair method here"""

        window_pred_prices = dict(
            zip(np.arange(0, 24), window_prices["value"].tolist())
        )

        current_prices = window_pred_prices.copy()
        #print(current_prices)
        it = 0

        qty = dict(zip(np.arange(0, len(current_prices)), [self.unit_pwr]*len(current_prices)))
        current_diff = 0
        valid = False
        removed_prices_sell = []
        removed_prices_buy = []

        while valid is False:

            dict_res = self._pair_method_loop(current_prices, qty)
            passes_list, reason = dict_res.values()
            # if not gain at the ieme pass, we juste have to validate
            if reason == "diff" and passes_list["iteration"] == 0:
                # need to end at n-1 : already the case
                # need to validate!!
                log.info("Negative at the first pass")
                passes_list = {}
                break
            log.info(f"""PASSE OUT : {len(passes_list["passes"])}""")
            orders = self._get_action_window(passes_list, prices=window_pred_prices)
            log.info(f"""ORDERS : {len(orders)}""")

            dict_validity = self.validity_checker.check_orders_validty(
                orders, stock, balance
            )

            # check whether empty or full at ieme pass
            if dict_validity["reason"] == "empty":
                # print(f'STOCK EMPTY AT POS {dict_validity["idx"]}')
                # we search sell before buy and remove buy
                buy_idx = self.search_sBb(passes_list)
                if buy_idx is not None:
                    # reset prices
                    # print(f"sBb : {buy_idx} : {current_prices[buy_idx]}")
                    removed_prices_buy.append(buy_idx)
                    current_prices = window_pred_prices.copy()
                    current_prices = {
                        k: v
                        for k, v in current_prices.items()
                        if k not in (removed_prices_buy + removed_prices_sell)
                    }

            elif dict_validity["reason"] == "full":
                # print(f'STOCK FULL AT POS {dict_validity["idx"]}')
                # we search buy before sell and remove sell
                sell_idx = self.search_bBs(passes_list)
                if sell_idx is not None:
                    # reset prices
                    # print(f"bBs : {sell_idx} : {current_prices[sell_idx]}")
                    removed_prices_buy.append(sell_idx)
                    current_prices = window_pred_prices.copy()
                    current_prices = {
                        k: v
                        for k, v in current_prices.items()
                        if k not in (removed_prices_buy + removed_prices_sell)
                    }

            else:
                # valide
                valid = True
                # print("VALID TRADING DAY")
                # print(f"removed buy sell : {[(_,window_prices[_]) for _ in removed_prices_buy]}")
                # print(f"removed sell prices : {[(_,window_prices[_]) for _ in removed_prices_sell]}")

            it += 1

            # reset qty
            qty = dict(
                zip(list(current_prices.keys()), [self.unit_pwr]*len(current_prices))
            )
        if passes_list:
            orders = self._get_action_window(passes_list, prices=window_pred_prices)
        else:
            orders = pd.DataFrame(
                {
                    tuple([k, v, 0, TradeAction.HOLD.name])
                    for k, v in window_pred_prices.items()
                },
                columns=["hour", "value", "flow", "action"],
            )
        orders.index = window_prices.index
        orders = pd.concat([orders, window_prices["true_prices"]], axis=1)
        return orders

    def _pair_method_loop(self, current_prices, qty):
        """
        Define "while" loop for pair method (set of "passes") :
            - find min (idx, value) for a qty
            - find max (up to 2 )(idx, value) based on rest qty to sell
            - break on negative diff between sell and buy
        """

        current_diff = 0
        passes_list = self.init_passes_list()
        pos = 1
        while (
            (current_diff >= 0)
            and (len(current_prices) > 1)
            and (passes_list["iteration"] < 24)
        ):
            it = passes_list["iteration"]
            print(f"--- PASSE : {it+1} ----")
            passe = self.init_passe()

            sell_order = 0
            buy_order = 0

            if it > 0:
                current_prices = passes_list["passes"][-1]["prices"].copy()

            #
            # BUY
            #

            min_p = self.find_min(current_prices)

            passe["min"] = {
                "qty": self.unit_pwr * 1.0,
                "value": min_p,
            }
            buy_order += min_p.value * passe["min"]["qty"]

            del current_prices[min_p.idx]

            #
            # SELL
            #

            max_p = self.find_max(current_prices)
            # we still have available max value to sell
            if passes_list["still_cpty"] > 0:
                passe["max"] = {
                    # 1h
                    "qty": np.round(
                        min(
                            self.unit_pwr * 1 * self.rho_d * self.rho_s,
                            passes_list["still_cpty"],
                        ),
                        3,
                    ),
                    "value": max_p,
                }

                qty[max_p.idx] = np.round(qty[max_p.idx] - passe["max"]["qty"], 3)

                passes_list["still_cpty"] -= passe["max"]["qty"]

                # we can sell a qty of second max value
                sell_order += max_p.value * passe["max"]["qty"]

                if qty[max_p.idx] <= 0:
                    pos = 0
                    print("DEL")
                    del current_prices[max_p.idx]

                if len(current_prices) == 0:
                    #
                    # CHANGE HERE
                    #
                    return dict(passes=passes_list, reason="empty prices list")

                if not self.check_max_sell(passe["max"]["qty"]):
                    # print(current_prices)
                    max_p = self.find_max(current_prices, pos=pos)
                    # reset position for second maxixum
                    pos = 1

                    passe["max2"] = {
                        # 1h
                        "qty": np.round(
                            np.round(self.unit_pwr * 1.0 * self.rho_d * self.rho_s, 1)
                            - passe["max"]["qty"],
                            1,
                        ),
                        "value": max_p,
                    }
                    qty[max_p.idx] = np.round(qty[max_p.idx] - passe["max2"]["qty"], 1)
                    sell_order += max_p.value * passe["max2"]["qty"]

                    passes_list["still_cpty"] = (
                        self.unit_pwr * 1.0 - passe["max2"]["qty"]
                    )

                    if qty[max_p.idx] <= 0:
                        del current_prices[max_p.idx]

            else:
                passe["max"] = {
                    "qty": np.round(self.unit_pwr * 1.0 * self.rho_d * self.rho_s, 1),
                    "value": max_p,
                }
                passes_list["still_cpty"] = self.unit_pwr - (
                    np.round(self.unit_pwr * 1.0 * self.rho_d * self.rho_s, 1)
                )

                sell_order += max_p.value * passe["max"]["qty"]
                qty[max_p.idx] = np.round(qty[max_p.idx] - passe["max"]["qty"], 1)

                if qty[max_p.idx] <= 0:
                    del current_prices[max_p.idx]
            # --- info ---
            print(f"""SELL : {passe["max"]["value"]} -- qty : {passe["max"]["qty"]}""")
            print(
                f"""SELL : {passe["max2"]["value"]} -- qty : {passe["max2"]["qty"]}"""
            )
            print(f"""BUY : {passe["min"]["value"]} -- qty : {passe["min"]["qty"]}""")

            ##
            # GAIN
            ##
            passe["diff"] = np.round(sell_order - buy_order, 1)
            current_diff = passe["diff"]

            passe["prices"] = current_prices
            # print(current_prices)

            if passe["diff"] < 0:
                print(f"""OUT : {passe["diff"]} => back passe {it}""")
                # print(f"""RESTE : {passes_list["still_cpty"]}""")
                return dict(passes=passes_list, reason="diff")
            passes_list["passes"].append(passe)

            passe["prices"]
            passes_list["iteration"] += 1

        return dict(passes=passes_list, reason="normal")

    def check_max_sell(self, qty):
        return qty >= np.round(self.unit_pwr * self.rho_d * self.rho_s, 1)

    def _get_action_window(self, passes_list, prices):
        """associate buy/sell/hold value to each hour and price"""

        it = passes_list["iteration"]
        sell_order = [
            tuple(
                [
                    _["max"]["value"].idx,
                    _["max"]["value"].value,
                    _["max"]["qty"],
                    _["max"]["value"].type_order,
                ]
            )
            for _ in passes_list["passes"]
        ]
        sell2_order = [
            tuple(
                [
                    _["max2"]["value"].idx,
                    _["max2"]["value"].value,
                    _["max2"]["qty"],
                    _["max2"]["value"].type_order,
                ]
            )
            for _ in passes_list["passes"]
            if isinstance(_["max2"]["value"], Price)
        ]
        buy_order = [
            tuple(
                [
                    _["min"]["value"].idx,
                    _["min"]["value"].value,
                    _["min"]["qty"],
                    _["min"]["value"].type_order,
                ]
            )
            for _ in passes_list["passes"]
        ]
        sell_order = sell2_order + sell_order
        orders = pd.DataFrame(
            sell_order + buy_order, columns=["hour", "value", "flow", "action"]
        )
        orders = (
            orders.groupby(["hour", "value"])
            .agg(flow=("flow", sum), action=("action", "first"))
            .reset_index()
        )
        hold_orders = pd.DataFrame(
            [
                tuple([k, v, 0, TradeAction["HOLD"].name])
                for k, v in prices.items()
                if k not in list(orders["hour"])
            ],
            columns=["hour", "value", "flow", "action"],
        )

        orders = pd.concat([orders, hold_orders]).set_index("hour").sort_index()
        # res.reindex(full_idx)
        return orders

    def search_sBb(self, passes_list):
        """return the first pass (frop the end) we sell before buy"""
        lenght = len(passes_list["passes"][::-1])

        for idx, passe in enumerate(passes_list["passes"][::-1]):
            sell_idx = (
                min(passe["max"]["value"].idx, passe["max2"]["value"].idx)
                if isinstance(passe["max2"]["value"], Price)
                else passe["max"]["value"].idx
            )
            buy_idx = passe["min"]["value"].idx
            if sell_idx < buy_idx:
                # print(f"FIND sBb FOR PASSE {lenght-idx} ")
                # idx = len(passes_list["passes"]) - idx
                return buy_idx
        return None

    def search_bBs(self, passes_list):
        """return the first pass (frop the end) we buy before sell"""

        for idx, passe in enumerate(passes_list["passes"][::-1]):
            sell_idx = (
                min(passe["max"]["value"].idx, passe["max2"]["value"].idx)
                if isinstance(passe["max2"]["value"], Price)
                else passe["max"]["value"].idx
            )
            buy_idx = passe["min"]["value"].idx
            if buy_idx < sell_idx:
                # idx = len(passes_list["passes"]) - idx
                return sell_idx
        return None

    def init_passe(self):
        return {
            "max": {
                "qty": 0,
                "value": 0,
            },
            "max2": {
                "qty": 0,
                "value": 0,
            },
            "min": {
                "qty": 0,
                "value": 0,
            },
            "diff": None,
        }

    def init_passes_list(self):
        return {
            "still_cpty": 0,
            "passes": [],
            "stock": [self.night_stock],
            "gain": [],
            "iteration": 0,
        }

    def find_max(self, prices, pos=0):
        max_ = sorted(prices.items(), key=lambda x: (x[1], [0]), reverse=True)[pos]
        return Price(idx=max_[0], value=max_[1], type_order=TradeAction["SELL"].name)

    def find_min(self, prices, pos=0):
        min_ = sorted(prices.items(), key=lambda x: (x[1], [0]), reverse=False)[pos]
        return Price(idx=min_[0], value=min_[1], type_order=TradeAction["BUY"].name)

    def test_loop(self, window_prices):
        current_prices = window_prices.copy()
        qty = dict(zip(np.arange(0, len(current_prices)), np.ones(len(current_prices))))
        dict_res = self._pair_method_loop(current_prices, qty)
        return dict_res


In [158]:
unit_pwr = 100
config_exp["rho_s"] = np.sqrt(0.7)
config_exp["rho_d"] = np.sqrt(0.7)

init_storage_cpty = 300

stock = Stock(storage_pwr=unit_pwr, 
                rho_d=config_exp["rho_d"], 
                rho_s=config_exp["rho_s"],
                t_discharge=config_exp["t_discharge"],
                init_storage_cpty=init_storage_cpty,
                )

balance = Balance(init_level=config_exp["balance_init_level"])

pair_method = PairMethod(storage_pwr=unit_pwr, 
           rho_s=config_exp["rho_s"], 
           rho_d=config_exp["rho_d"], 
           init_storage_cpty=init_storage_cpty)

In [159]:
prices

,true_prices,value
2020-01-03 00:00:00,13.04,18.400147
2020-01-03 01:00:00,1.53,15.482259
2020-01-03 02:00:00,0.14,12.918863
2020-01-03 03:00:00,0.85,12.865515
2020-01-03 04:00:00,9.92,13.802502
2020-01-03 05:00:00,24.48,16.094563
2020-01-03 06:00:00,26.68,20.752541
2020-01-03 07:00:00,28.81,24.541543
2020-01-03 08:00:00,29.28,29.486725
2020-01-03 09:00:00,28.85,32.287523


In [160]:
res = pair_method.get_action_window(prices, stock, balance)

INFO:__main__:PASSE OUT : 10
INFO:__main__:ORDERS : 24
INFO:__main__:ADD : -1840.014679941871 €
INFO:__main__:Current : -840.014679941871 €
INFO:__main__:ADD : -1548.225861396329 €
INFO:__main__:Current : -2388.2405413382003 €
INFO:__main__:ADD : -1291.886287991087 €
INFO:__main__:Current : -3680.1268293292874 €
INFO:__main__:ADD : -1286.551509389429 €
INFO:__main__:Current : -4966.678338718716 €
INFO:__main__:ADD : -1380.250153261935 €
INFO:__main__:Current : -6346.928491980651 €
INFO:__main__:ADD : -1609.4563002822 €
INFO:__main__:Current : -7956.384792262851 €
INFO:__main__:ADD : -2075.254083373805 €
INFO:__main__:Current : -10031.638875636656 €
INFO:__main__:ADD : -2454.154316677214 €
INFO:__main__:Current : -12485.79319231387 €
INFO:__main__:PASSE OUT : 9
INFO:__main__:ORDERS : 24
INFO:__main__:ADD : -1840.014679941871 €
INFO:__main__:Current : -840.014679941871 €
INFO:__main__:ADD : -1548.225861396329 €
INFO:__main__:Current : -2388.2405413382003 €
INFO:__main__:ADD : -1291.88628

--- PASSE : 1 ----
SELL : Price(idx=16, value=41.52084675735035, type_order='SELL') -- qty : 70.0
SELL : 0 -- qty : 0
BUY : Price(idx=3, value=12.86551509389429, type_order='BUY') -- qty : 100.0
--- PASSE : 2 ----
DEL
SELL : Price(idx=16, value=41.52084675735035, type_order='SELL') -- qty : 30.0
SELL : Price(idx=15, value=41.14980032777058, type_order='SELL') -- qty : 40.0
BUY : Price(idx=2, value=12.91886287991087, type_order='BUY') -- qty : 100.0
--- PASSE : 3 ----
DEL
SELL : Price(idx=15, value=41.14980032777058, type_order='SELL') -- qty : 60.0
SELL : Price(idx=14, value=40.91954833355022, type_order='SELL') -- qty : 10.0
BUY : Price(idx=4, value=13.80250153261935, type_order='BUY') -- qty : 100.0
--- PASSE : 4 ----
SELL : Price(idx=14, value=40.91954833355022, type_order='SELL') -- qty : 70.0
SELL : 0 -- qty : 0
BUY : Price(idx=1, value=15.48225861396329, type_order='BUY') -- qty : 100.0
--- PASSE : 5 ----
DEL
SELL : Price(idx=14, value=40.91954833355022, type_order='SELL') -- qty

INFO:__main__:ADD : 4152.084675735035 €
INFO:__main__:Current : 6239.8186747276095 €
INFO:__main__:ADD : 4053.5873296583263 €
INFO:__main__:Current : 10293.406004385935 €
INFO:__main__:ADD : 0.0 €
INFO:__main__:Current : 10293.406004385935 €
INFO:__main__:ADD : 2058.456994497288 €
INFO:__main__:Current : 12351.862998883224 €
INFO:__main__:ADD : 0.0 €
INFO:__main__:Current : 12351.862998883224 €
INFO:__main__:ADD : 0.0 €
INFO:__main__:Current : 12351.862998883224 €
INFO:__main__:ADD : 0.0 €
INFO:__main__:Current : 12351.862998883224 €
INFO:__main__:ADD : -2141.5562208090973 €
INFO:__main__:Current : 10210.306778074126 €


SELL 100.0
=== 16 ===
Current cpty : 527.0934357735347
Current balance : 2087.733998992575
current : 527.0934357735347 - retrieve qty : 119.52286093343936 - flow : 100.0
Updated Stock : 407.57057484009533
SELL 100.0
=== 17 ===
Current cpty : 407.57057484009533
Current balance : 6239.8186747276095
current : 407.57057484009533 - retrieve qty : 119.52286093343936 - flow : 100.0
Updated Stock : 288.047713906656
SELL 100.0
=== 18 ===
Current cpty : 288.047713906656
Current balance : 10293.406004385935
Updated Stock : 288.047713906656
HOLD 0
=== 19 ===
Current cpty : 288.047713906656
Current balance : 10293.406004385935
current : 288.047713906656 - retrieve qty : 71.71371656006362 - flow : 60.0
Updated Stock : 216.33399734659235
SELL 60.0
=== 20 ===
Current cpty : 216.33399734659235
Current balance : 12351.862998883224
Updated Stock : 216.33399734659235
HOLD 0
=== 21 ===
Current cpty : 216.33399734659235
Current balance : 12351.862998883224
Updated Stock : 216.33399734659235
HOLD 0
=== 22 ==

In [154]:
config_exp["balance_init_level"]

1000

In [155]:
res.loc[res.action == "BUY", "flow"] = res.loc[res.action == "BUY", "flow"]*-1

In [156]:
res

,value,flow,action,true_prices
2020-01-03 00:00:00,18.400147,-100.0,BUY,13.04
2020-01-03 01:00:00,15.482259,-100.0,BUY,1.53
2020-01-03 02:00:00,12.918863,-100.0,BUY,0.14
2020-01-03 03:00:00,12.865515,-100.0,BUY,0.85
2020-01-03 04:00:00,13.802502,-100.0,BUY,9.92
2020-01-03 05:00:00,16.094563,-100.0,BUY,24.48
2020-01-03 06:00:00,20.752541,-100.0,BUY,26.68
2020-01-03 07:00:00,24.541543,0.0,HOLD,28.81
2020-01-03 08:00:00,29.486725,0.0,HOLD,29.28
2020-01-03 09:00:00,32.287523,0.0,HOLD,28.85


In [150]:
stock

Stock(storage_pwr=100, rho_d=0.8366600265340756, rho_s=0.8366600265340756, t_discharge=8, init_storage_cpty=300, current_cpty=300, storage_cpty=956)

In [148]:
(res["true_prices"]*res["flow"]).sum()

11407.4

In [149]:
(res["value"]*res["flow"]).sum()

9210.306778074128

### Update stock and balance

In [23]:
stock_c = copy.copy(stock)
balance_c = copy.copy(balance)

for _, s_price in res.iterrows():
    print(_)
    flow = stock_c.update_stock(s_price['action'], s_price["flow"])

    balance_c.update_balance(action=s_price['action'], 
                            price=s_price['true_prices'],
                            flow=flow,
                            )

INFO:__main__:ADD : 0.0 €
INFO:__main__:Current : 1000.0 €
INFO:__main__:ADD : 0.0 €
INFO:__main__:Current : 1000.0 €
INFO:__main__:ADD : -46.6 €
INFO:__main__:Current : 953.4 €
INFO:__main__:ADD : -46.72 €
INFO:__main__:Current : 906.68 €
INFO:__main__:ADD : -47.58 €
INFO:__main__:Current : 859.0999999999999 €
INFO:__main__:ADD : 0.0 €
INFO:__main__:Current : 859.0999999999999 €
INFO:__main__:ADD : 73.06 €
INFO:__main__:Current : 932.1599999999999 €
INFO:__main__:ADD : 79.0 €
INFO:__main__:Current : 1011.1599999999999 €
INFO:__main__:ADD : 75.65 €
INFO:__main__:Current : 1086.81 €
INFO:__main__:ADD : 72.03 €
INFO:__main__:Current : 1158.84 €
INFO:__main__:ADD : 13.278 €
INFO:__main__:Current : 1172.118 €
INFO:__main__:ADD : 0.0 €
INFO:__main__:Current : 1172.118 €
INFO:__main__:ADD : 0.0 €
INFO:__main__:Current : 1172.118 €
INFO:__main__:ADD : 0.0 €
INFO:__main__:Current : 1172.118 €
INFO:__main__:ADD : 0.0 €
INFO:__main__:Current : 1172.118 €
INFO:__main__:ADD : 0.0 €
INFO:__main__:C

2020-01-02 00:00:00
Updated Stock : 3.0
2020-01-02 01:00:00
Updated Stock : 3.0
2020-01-02 02:00:00
current : 3.0 - add qty : 0.84 - flow : -1.0
Updated Stock : 3.84
2020-01-02 03:00:00
current : 3.84 - add qty : 0.84 - flow : -1.0
Updated Stock : 4.68
2020-01-02 04:00:00
current : 4.68 - add qty : 0.84 - flow : -1.0
Updated Stock : 5.52
2020-01-02 05:00:00
Updated Stock : 5.52
2020-01-02 06:00:00
current : 5.52 - retrieve qty : 1.1904761904761905 - flow : 1.0
Updated Stock : 4.329523809523809
2020-01-02 07:00:00
current : 4.329523809523809 - retrieve qty : 1.1904761904761905 - flow : 1.0
Updated Stock : 3.139047619047618
2020-01-02 08:00:00
current : 3.139047619047618 - retrieve qty : 1.1904761904761905 - flow : 1.0
Updated Stock : 1.9485714285714277
2020-01-02 09:00:00
current : 1.9485714285714277 - retrieve qty : 1.1904761904761905 - flow : 1.0
Updated Stock : 0.7580952380952373
2020-01-02 10:00:00
current : 0.7580952380952373 - retrieve qty : 0.2380952380952381 - flow : 0.2
Updated

In [48]:
(res[res.action == "SELL"].value.sum()) - (res[res.action == "BUY"].value.sum())

107.77000000000004

In [265]:
balance.current_level

1000